<a href="https://colab.research.google.com/github/nasif-raihan/ML-and-DL-Codes/blob/main/Model_Pipeline.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import pandas as pd
import seaborn as sns

In [2]:
from sklearn.model_selection import train_test_split
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import MinMaxScaler, OneHotEncoder
from sklearn.pipeline import Pipeline
from sklearn.feature_selection import SelectKBest, chi2
from sklearn.tree import DecisionTreeClassifier

In [3]:
df = sns.load_dataset('titanic')
print(f"{df.shape = }")
df.sample(2)

df.shape = (891, 15)


,survived,pclass,sex,age,sibsp,parch,fare,embarked,class,who,adult_male,deck,embark_town,alive,alone
395,0,3,male,22.0,0,0,7.7958,S,Third,man,True,NaN,Southampton,no,True
421,0,3,male,21.0,0,0,7.7333,Q,Third,man,True,NaN,Queenstown,no,True


In [4]:
# dropping unnecessary columns for prediction
df.drop(columns=['class', 'who', 'adult_male', 'deck', 'alive', 'alone', 'embark_town'], inplace=True)

print(f"{df.shape = }\n")
df.head(2)

df.shape = (891, 8)



,survived,pclass,sex,age,sibsp,parch,fare,embarked
0,0,3,male,22.0,1,0,7.2500,S
1,1,1,female,38.0,1,0,71.2833,C


In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 8 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   survived  891 non-null    int64  
 1   pclass    891 non-null    int64  
 2   sex       891 non-null    object 
 3   age       714 non-null    float64
 4   sibsp     891 non-null    int64  
 5   parch     891 non-null    int64  
 6   fare      891 non-null    float64
 7   embarked  889 non-null    object 
dtypes: float64(2), int64(4), object(2)
memory usage: 55.8+ KB


In [6]:
df.isnull().sum()

survived      0
pclass        0
sex           0
age         177
sibsp         0
parch         0
fare          0
embarked      2
dtype: int64

In [7]:
# split into train and test dataset
X = df.drop(columns=['survived'])
y = df['survived']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

print(f"{X_train.shape = }")
print(f"{X_test.shape = }")

X_train.shape = (712, 7)
X_test.shape = (179, 7)


In [8]:
X_train.head(1)

,pclass,sex,age,sibsp,parch,fare,embarked
331,1,male,45.5,0,0,28.5,S


In [9]:
feature_names = X_train.columns
print(f'{feature_names = }')

feature_names = Index(['pclass', 'sex', 'age', 'sibsp', 'parch', 'fare', 'embarked'], dtype='object')


In [10]:
y_train.head(1)

331    0
Name: survived, dtype: int64

In [11]:
X_train.sample(1)

,pclass,sex,age,sibsp,parch,fare,embarked
348,3,male,3.0,1,1,15.9,S


# Pipeline

In [12]:
# imputation transformer
trf1 = ColumnTransformer(transformers=[
    ('impute_age', SimpleImputer(), [2]),
    ('impute_embarked', SimpleImputer(strategy='most_frequent'), [6])
], remainder='passthrough')

In [13]:
# one hot encoding
trf2 = ColumnTransformer(transformers=[
    ('ohe_sex_embarked', OneHotEncoder(sparse_output=False, handle_unknown='ignore'), [1, 6]),
], remainder='passthrough')

In [14]:
# scaling
trf3 = ColumnTransformer(transformers=[
    ('scale', MinMaxScaler(), slice(0, 10))
])

In [15]:
# feature selection
trf4 = SelectKBest(score_func=chi2, k=8)

In [16]:
# define model
trf5 = DecisionTreeClassifier()

In [17]:
pipe = Pipeline(
    [
        ('trf1',trf1),
        ('trf2',trf2),
        ('trf3',trf3),
        ('trf4',trf4),
        ('trf5',trf5)
    ]
)

In [18]:
# train
pipe.fit(X_train, y_train)

Pipeline(steps=[('trf1',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('impute_age', SimpleImputer(),
                                                  [2]),
                                                 ('impute_embarked',
                                                  SimpleImputer(strategy='most_frequent'),
                                                  [6])])),
                ('trf2',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('ohe_sex_embarked',
                                                  OneHotEncoder(handle_unknown='ignore',
                                                                sparse_output=False),
                                                  [1, 6])])),
                ('trf3',
                 ColumnTransformer(transformers=[('scale', MinMaxScaler(),
                                                  slice(0, 10, None))])),
                ('trf4',
                 SelectKBest(k=8,
                             score_func=<function chi2 at 0x7fba36f00c10>)),
                ('trf5', DecisionTreeClassifier())])

In [19]:
from sklearn.metrics import accuracy_score


# Predict
y_pred = pipe.predict(X_test)
accuracy_score(y_test, y_pred)

0.6256983240223464

# Cross Validation using Pipeline

In [20]:
from sklearn.model_selection import cross_val_score

cross_val_score(pipe, X_train, y_train, cv=5, scoring='accuracy').mean()

0.6391214419383433

# GridSearch using Pipeline

In [21]:
# gridsearchcv
params = {
    'trf5__max_depth':[1,2,3,4,5,None]
}

In [22]:

from sklearn.model_selection import GridSearchCV

grid = GridSearchCV(pipe, params, cv=5, scoring='accuracy')
grid.fit(X_train, y_train)

GridSearchCV(cv=5,
             estimator=Pipeline(steps=[('trf1',
                                        ColumnTransformer(remainder='passthrough',
                                                          transformers=[('impute_age',
                                                                         SimpleImputer(),
                                                                         [2]),
                                                                        ('impute_embarked',
                                                                         SimpleImputer(strategy='most_frequent'),
                                                                         [6])])),
                                       ('trf2',
                                        ColumnTransformer(remainder='passthrough',
                                                          transformers=[('ohe_sex_embarked',
                                                                         OneHotEncoder(handle_unknown='ignore',
                                                                                       sparse_output=False),
                                                                         [1,
                                                                          6])])),
                                       ('trf3',
                                        ColumnTransformer(transformers=[('scale',
                                                                         MinMaxScaler(),
                                                                         slice(0, 10, None))])),
                                       ('trf4',
                                        SelectKBest(k=8,
                                                    score_func=<function chi2 at 0x7fba36f00c10>)),
                                       ('trf5', DecisionTreeClassifier())]),
             param_grid={'trf5__max_depth': [1, 2, 3, 4, 5, None]},
             scoring='accuracy')

# Exporting the Pipeline

In [23]:
import pickle

pickle.dump(pipe, open('pipe-dt.pkl', 'wb'))

# How to use the model in production?

In [24]:
pipe = pickle.load(open('pipe-dt.pkl', 'rb'))

In [25]:
print(f'{feature_names = }')

feature_names = Index(['pclass', 'sex', 'age', 'sibsp', 'parch', 'fare', 'embarked'], dtype='object')


In [26]:
# Assume user input
input = np.array([2, 'male', 31.0, 0, 0, 10.5, 'S'],dtype=object).reshape(1,7)
input.shape

(1, 7)

In [27]:
test_df = pd.DataFrame(input.tolist(), columns=feature_names)
test_df.head()

,pclass,sex,age,sibsp,parch,fare,embarked
0,2,male,31.0,0,0,10.5,S


In [28]:
pipe.predict(test_df)

array([0])